## The API Handler

<br>
<figure>
 <img src="assets/prompt.png" width="100%" align="center"/></a>
<figcaption> SQL AI Agent Architecture </figcaption>
</figure>

<br>

Below is the `create_message` function blueprint. It has the following two arguments:
- tbl_name - the table name
- user_request - the user request or question

It returns an object with the user and system prompts.


<br>
<figure>
 <img src="assets/prompt_handler.png" width="70%" align="center"/></a>
<figcaption> The Prompt Handler </figcaption>
</figure>

<br>

In [1]:
import sys
import os
import pandas as pd
import duckdb as db
sys.path.append("../")
from sql_ai_agent import api_handler as ah

In [2]:
base_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
gemini_api_key = os.environ.get("GEMINI_API_KEY")

In [3]:
retail_sales = pd.read_csv("../data/Red30 Tech US online retail sales.csv")

db.register("retail_sales", retail_sales)


In [4]:
sql_agent = ah.SqlAgent(api_key = gemini_api_key, 
                        base_url = base_url, 
                        model = "gemini-2.5-pro", 
                        tbl_name = "retail_sales",
                        max_token= 5000)



In [5]:
print(sql_agent.system.system)


Given the following SQL table, your job is to write queries given a user’s request. Return just the SQL query as plain text, without additional text, and don't use markdown format.

CREATE TABLE retail_sales (OrderNum BIGINT, OrderDate VARCHAR, OrderType VARCHAR, CustomerType VARCHAR, CustName VARCHAR, CustState VARCHAR, ProdCategory VARCHAR, ProdNumber VARCHAR, ProdName VARCHAR, Quantity BIGINT, Price DOUBLE, Discount DOUBLE, OrderTotal DOUBLE)



In [6]:
question = "What are the total sales by state?"

sql_agent.send_prompt(question=question)

In [7]:
print(sql_agent.query)

SELECT CustState, SUM(OrderTotal) FROM retail_sales GROUP BY CustState


In [8]:
sql_agent.ask_question(question=question)

In [9]:
sql_agent.data

┌────────────────┬────────────────────┐
│   CustState    │  sum(OrderTotal)   │
│    varchar     │       double       │
├────────────────┼────────────────────┤
│ Texas          │ 349925.48000000004 │
│ West Virginia  │           23967.97 │
│ Alabama        │ 137984.65000000005 │
│ South Carolina │  55752.65999999998 │
│ New Jersey     │            35971.9 │
│ Tennessee      │ 206698.71999999994 │
│ Oregon         │ 113873.62000000001 │
│ Nebraska       │           22604.36 │
│ North Dakota   │            29907.5 │
│ VIrginia       │              189.0 │
│    ·           │                ·   │
│    ·           │                ·   │
│    ·           │                ·   │
│ Delaware       │           38611.46 │
│ Maine          │               49.0 │
│ Louisiana      │ 113639.52999999996 │
│ Washington     │ 257140.79999999993 │
│ Connecticut    │ 57114.099999999984 │
│ Montana        │ 51841.609999999986 │
│ New York       │  616925.8400000001 │
│ Colorado       │ 174726.32000000004 │
